## Example of extracting the lever arm and the charging energy from bias triangles and addition lines

Authors: Anne-Marije Zwerver and Pieter Eendebak

Import the modules used in this program:

In [1]:
%pylab tk
#%matplotlib inline
import os, sys
import qcodes
import qtt
import matplotlib.pyplot as plt
import numpy as np

from qcodes import MatPlot
from qtt.data import diffDataset
from qtt.algorithms.bias_triangles import perpLineIntersect, lever_arm, E_charging

Populating the interactive namespace from numpy and matplotlib


### Load datasets

In [2]:
#exampledatadir = os.path.join(qtt.__path__[0],'docs', 'notebooks', 'data')
#exampledatadir = os.path.join(os.getcwd(), 'data', 'Lever_arm_Charging_energy')
#qcodes.DataSet.default_io = qcodes.DiskIO(exampledatadir)
#dataset_la = qcodes.load_data('14-33-26_qtt_scan2Dfast')
#dataset_Ec = qcodes.load_data('10-06-59_qtt_scan2Dfast')

exampledatadir=os.path.join(qtt.__path__[0],'..', 'docs', 'notebooks','data')
qcodes.DataSet.default_io = qcodes.DiskIO(exampledatadir)
dataset_la = qcodes.load_data(os.path.join(exampledatadir, r'Lever_arm_Charging_energy\14-33-26_qtt_scan2Dfast'))
dataset_Ec = qcodes.load_data(os.path.join(exampledatadir, r'Lever_arm_Charging_energy\10-06-59_qtt_scan2Dfast'))

Plot the bias triangles (to extract the lever arm)

In [3]:
plt.figure(1); plt.clf()
MatPlot([dataset_la.measured], num = 1)
ax = plt.gca()
plt.suptitle('Bias triangle') 

Plot the addition lines (to extraxt the charging energy)

In [4]:
plt.figure(3); plt.clf()
MatPlot([dataset_Ec.measured], num = 3)
ax = plt.gca()
plt.suptitle('Addition lines') 

### Extract lever arm from the bias triangles

First, click three points on the figure. Point 1 and 2 along the addition line for the dot of which you want to determine the lever arm, the third point on the triple point where both dot levels are aligned. The perpLineIntersect function will return a dictionary containing the coordinates of these three points, the intersection point of a horizontal/vertical line of the third point with the (extended) line through point 1 and 2 and the line length from the third point to the intersection.

If you want to determine the lever arm for dot 4, take the horizontal line (vertical = False),
if you want to determine the lever arm for dot 5, take the vertical line (vertical = True).

In [6]:
dot = 'P5'

if dot == 'P4':
    vertical = False
elif dot == 'P5':
    vertical = True
else:
    print("Please choose either dot 4 or dot 5")

la_fit = perpLineIntersect(dataset_la, vertical = vertical, description = 'lever_arm')

Please click three points;
            Point 1: on the addition line for the dot represented on the vertical axis
            Point 2: further on the addition line for the dot represented on the vertical axis
            Point 3: on the triple point at the addition line for the dot represented on the horizontal axis
            where both dot levels are aligned


C:\Users\TUD278090\AppData\Local\Continuum\Anaconda3\envs\amjzwerver\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Determine the lever arm ($\mu$V/mV) by dividing the applied bias for the bias triangles by the voltage span determined by la_fit

In [10]:
bias = dataset_la.snapshot()['allgatevalues']['O5']
lev_arm = lever_arm(bias, la_fit, fig = True)
print('''The lever arm of dot %s is %.2f ueV/mV'''%(dot, lev_arm))

The lever arm of dot P5 is 48.69 ueV/mV


### Extract addition energy

Again, use the function perpLineIntersect, this time to obtain the charging energy (in mV). By means of the lever arm, the chagring energy can be expressed in meV

In [11]:
Ec_fit = perpLineIntersect(dataset_Ec, description = 'E_charging', vertical = vertical)

Please click three points;
            Point 1: on the (0, 1) - (0,2) addition line
            Point 2: further on the (0, 1) - (0,2) addition line
            Point 3: on the (0, 0) - (0, 1) addition line 


C:\Users\TUD278090\AppData\Local\Continuum\Anaconda3\envs\amjzwerver\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [13]:
E_c = E_charging(lev_arm, results = Ec_fit, fig = True)
print('''The charging energy for dot %s is %.2f meV'''%(dot, E_c/1000))

The charging energy for dot P5 is 2.50 meV
